# Function calling

In [ ]:
import openai
import os
import json
import requests
from dotenv import load_dotenv

In [2]:
load_dotenv()  # Load environment variables from .env file

# Ensure the OPENAI_API_KEY is set
openai.api_key = os.getenv("OPENAI_API_KEY")
if openai.api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set")

client = openai.OpenAI()

## Get weather

### Setting up the Open-meteo API 
https://open-meteo.com/en/docs

In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache

from retry_requests import retry

In [4]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

Testing...

In [9]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -8.0498,
	"longitude": -34.8975,
	"hourly": "temperature_2m",
	"timezone": "auto",
	"forecast_days": 1
}
responses = openmeteo.weather_api(url, params=params)

In [10]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates -8.0°N -34.875°E
Elevation 9.0 m asl
Timezone b'America/Recife'b'GMT-3'
Timezone difference to GMT+0 -10800 s


In [11]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

,date,temperature_2m
0,2025-06-06 03:00:00+00:00,23.754499
1,2025-06-06 04:00:00+00:00,23.604500
2,2025-06-06 05:00:00+00:00,23.454500
3,2025-06-06 06:00:00+00:00,23.104500
4,2025-06-06 07:00:00+00:00,22.904501
5,2025-06-06 08:00:00+00:00,22.804501
6,2025-06-06 09:00:00+00:00,22.954500
7,2025-06-06 10:00:00+00:00,24.154501
8,2025-06-06 11:00:00+00:00,25.454500
9,2025-06-06 12:00:00+00:00,26.754499


### Functions to call

In [ ]:
# Function to get weather data using Open-Meteo API
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m&timezone=auto")
    data = response.json()
    return data['current']['temperature_2m']


# Route each function call to the appropriate function
available_functions = {
    "get_weather": get_weather,
    # "other_function": other_function,  # Uncomment and define other functions as needed
}

def call_function(name, args):
    if name in available_functions:
        return available_functions[name](**args)
    else:
        raise ValueError(f"Function {name} is not available.")

In [ ]:
# Specify the available tools for the model
tools = [{
    "type": "function",
    "name": "get_weather",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Paris and in San Francisco today?"}]

response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools,
)
response.output

[ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_0wIRj0NghaA7o8ZJKyKgklnx', name='get_weather', type='function_call', id='fc_68435e3edfd8819a96fbe4c7eadfcfbb019487faac95fa8e', status='completed'),
 ResponseFunctionToolCall(arguments='{"latitude":37.7749,"longitude":-122.4194}', call_id='call_1FvTI2y6P3zGWbKEVjEmc5cd', name='get_weather', type='function_call', id='fc_68435e3f08a0819aacddf3f3c723954a019487faac95fa8e', status='completed')]

When the model calls a function, you must execute it and return the result. Since model responses can include zero, one, or multiple calls, it is best practice to assume there are several.

In [34]:
# Iterate through the tool calls in the response
for tool_call in response.output:
    if tool_call.type != "function_call":
        continue

    # Append model's function call message
    input_messages.append(tool_call.to_dict())

    # Call the function with the provided arguments
    name = tool_call.name
    args = json.loads(tool_call.arguments)
    result = call_function(name, args)
    
    # Append function call result message
    input_messages.append({
        "type": "function_call_output",
        "call_id": tool_call.call_id,
        "output": str(result)
    })
input_messages

[{'role': 'user',
  'content': "What's the weather like in Paris and in San Francisco today?"},
 {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
  'call_id': 'call_0wIRj0NghaA7o8ZJKyKgklnx',
  'name': 'get_weather',
  'type': 'function_call',
  'id': 'fc_68435e3edfd8819a96fbe4c7eadfcfbb019487faac95fa8e',
  'status': 'completed'},
 {'type': 'function_call_output',
  'call_id': 'call_0wIRj0NghaA7o8ZJKyKgklnx',
  'output': '17.1'},
 {'arguments': '{"latitude":37.7749,"longitude":-122.4194}',
  'call_id': 'call_1FvTI2y6P3zGWbKEVjEmc5cd',
  'name': 'get_weather',
  'type': 'function_call',
  'id': 'fc_68435e3f08a0819aacddf3f3c723954a019487faac95fa8e',
  'status': 'completed'},
 {'type': 'function_call_output',
  'call_id': 'call_1FvTI2y6P3zGWbKEVjEmc5cd',
  'output': '19.2'}]

Incorporating results into response.

In [35]:
response = client.responses.create(
    model="gpt-4.1-nano",
    input=input_messages,
    tools=tools,
)
response.output_text

'The weather in Paris today is approximately 17.1°C, and in San Francisco, it is around 19.2°C.'